# Mnist cnn
Trains a simple convnet on the MNIST dataset. Reference: [Mnist cnn - Keras Documentation](https://keras.io/examples/mnist_cnn/)

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
K.image_data_format()

'channels_last'

In [6]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
keep_prob = 0.5
drop_rate = 1 - keep_prob

In [10]:
model = Sequential()

In [11]:
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate = drop_rate)) # keep_prob = keep_prob
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file="./mnist-model.png", show_shapes=True)

|Model Architecture|
|:--:|
|<img src="./mnist-model.png">|

In [14]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 156s 3ms/step - loss: 0.2704 - acc: 0.9153 - val_loss: 0.0624 - val_acc: 0.9789
Epoch 2/12
60000/60000 [==============================] - 160s 3ms/step - loss: 0.0900 - acc: 0.9737 - val_loss: 0.0406 - val_acc: 0.9867
Epoch 3/12
60000/60000 [==============================] - 146s 2ms/step - loss: 0.0682 - acc: 0.9799 - val_loss: 0.0315 - val_acc: 0.9884
Epoch 4/12
60000/60000 [==============================] - 157s 3ms/step - loss: 0.0553 - acc: 0.9832 - val_loss: 0.0301 - val_acc: 0.9894
Epoch 5/12
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0460 - acc: 0.9859 - val_loss: 0.0334 - val_acc: 0.9880
Epoch 6/12
60000/60000 [==============================] - 166s 3ms/step - loss: 0.0411 - acc: 0.9878 - val_loss: 0.0283 - val_acc: 0.9896
Epoch 7/12
60000/60000 [==============================] - 161s 3ms/step - 

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)

In [16]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.025654783184213374
Test accuracy: 0.9924


In [17]:
# Output as HDF5 format.
model.save('mnist_model.h5')

```bash
# Convert the model into tensorflow.js format.
tensorflowjs_converter --input_format keras mnist_model.h5 tf-model
```